In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pbo.environment.car_on_hill import CarOnHillEnv
import numpy as np

max_pos = 1.0
max_velocity = 3.0
gamma = 0.9


env = CarOnHillEnv(max_pos, max_velocity, gamma)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import jax.numpy as jnp 


env.reset()
absorbing = False
step = 0


while not absorbing and step < 15:
    env.render()

    _, reward, absorbing, _ = env.step(jnp.array([1]))
    step += 1

env.close()

## Time analysis

In [435]:
import jax 

def f(next_state):
    too_fast = (jnp.abs(next_state[1]) > env.max_velocity).astype(float)
    too_far_left = (next_state[0] < -env.max_pos).astype(float)
    too_far_right = (next_state[0] > env.max_pos).astype(float)

    return (jnp.minimum(too_far_left + too_fast, 1)) * jnp.array([-1.0, 1]) + (too_far_right * (1 - too_fast)) * jnp.array([1.0, 1])

jit_f = jax.jit(f)

In [443]:
import time
from scipy.integrate import odeint

action_input = jnp.array([1])

t0 = time.time()


reward, absording = f(env.state)

env.state, jnp.array([reward]), jnp.array([absorbing]), {}

print("500 steps:", 500 * (time.time() - t0) / 60)

500 steps: 0.025224685668945312


In [331]:
import 

NameError: name 'scipy' is not defined

In [ ]:
        action = self.actions_values[action[0]]
        sa = np.append(self.state, action)
        new_state = odeint(self._dpds, sa, [0, self._dt])

        self.state = jnp.array(new_state[-1, :-1])

        if self.state[0] < -self.max_pos or np.abs(self.state[1]) > self.max_velocity:
            reward = -1.0
            absorbing = True
        elif self.state[0] > self.max_pos and np.abs(self.state[1]) <= self.max_velocity:
            reward = 1.0
            absorbing = True
        else:
            reward = 0.0
            absorbing = False

        return self.state, jnp.array([reward]), jnp.array([absorbing]), {}